In [1]:
import os

import numpy as np
from sklearn.decomposition import PCA

from utils import load_hidden_representations_from_hdf5, read_templates_from_file

----

In [3]:
# params
log_dir = "/logfiles"
model = "bigscience-T0_3B" # bigscience-T0_B or bigscience-T0
module = "encoder" # encoder or decoder
task = "rte"

## Prompts

In [4]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}.csv")
display(df)

,name,template,category,shuffle
0,null_pattern,{premise} {hypothesis},neutral,False
1,null_pattern_reversed,{hypothesis} {premise},neutral,False
2,gpt_3_yes_no,{premise} Question: {hypothesis} Yes or No?,instructive,False
3,gpt_3_yes_no_shuffled,{premise} Question: {hypothesis} Yes or No?,instructive,True
4,gpt_3_true_false,{premise} Question: {hypothesis} True or False?,instructive,False
5,gpt_3_true_false_shuffled,{premise} Question: {hypothesis} True or False?,instructive,True
6,start_with_the,"{premise} Does the paragraph start with ""the""?...",misleading,False
7,mnli_crowdsource,{premise} Using only the above description and...,instructive,False
8,based_on_previous_passage,{premise} Based on the previous passage; is it...,instructive,False
9,infer,"Suppose {premise} Can we infer that ""{hypothes...",instructive,False


## PCA

In [5]:
def unison_shuffled_copies(a, b):
    # from: https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [10]:
# specify which patterns to use for classification
use_pattern = [
    # "null_pattern",
    # "null_pattern_reversed",
    "gpt_3_yes_no",
    # "gpt_3_yes_no_shuffled",
    # "gpt_3_true_false",
    # "gpt_3_true_false_shuffled",
    # "start_with_the",
    # "mnli_crowdsource",
    # "based_on_previous_passage",
    # "infer",
    # "follow",
    # "imply",
    # "guaranteed",
    # "justified", 
    # "must_be_true",
    "should_assume"
]

In [17]:
for layer in range(0, 3):
# for layer in range(0, 25):
# for layer in range(24, 25):

    file_names, prompt_names = [], []

    for _, row in df.iterrows():
        if row['name'] in use_pattern:
            file_names.append(f"rte/{model}/{module}/{row['name']}/hidden_represenations_layer{layer}_avg.hdf5",)
            prompt_names.append(row['name'])


    # load hidden representations from hdf5 file
    representations = None
    classes = []
    n_sequences = 0

    for idx, file_name in enumerate(file_names):
        hidden_representations = load_hidden_representations_from_hdf5(os.path.join(log_dir, file_name))
        # print(hidden_representations.shape)
        n_sequences = hidden_representations.shape[0]

        if representations is None:
            representations = hidden_representations
        else:
            representations = np.concatenate((representations, hidden_representations), axis=0)

        classes += n_sequences * [idx] # assign representations to classes
    
    classes = np.asarray(classes)

    # shuffle representations and classes
    X, y = unison_shuffled_copies(representations, classes)
    print(X.shape, y.shape)

    # perform PCA on hidden representations
    print('PCA for prompts:', prompt_names)

    for n_components in range(1, 5):
        pca = PCA(n_components=n_components)
        pca.fit(X)

        # variance explained by each of the principal components
        print(f"model:{model}; module:{module}; layer:{layer}; n_components: {n_components}; variance explained: {pca.explained_variance_ratio_}")
        print(np.sum(pca.explained_variance_ratio_))


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 4980.97it/s]


(554, 2048) (554,)
PCA for prompts: ['gpt_3_yes_no', 'should_assume']
model:bigscience-T0_3B; module:encoder; layer:0; n_components: 1; variance explained: [0.9136936]
0.9136936
model:bigscience-T0_3B; module:encoder; layer:0; n_components: 2; variance explained: [0.91369355 0.00523294]
0.9189265
model:bigscience-T0_3B; module:encoder; layer:0; n_components: 3; variance explained: [0.91369355 0.00523293 0.00414104]
0.9230675
model:bigscience-T0_3B; module:encoder; layer:0; n_components: 4; variance explained: [0.9136936  0.00523293 0.00414104 0.00295302]
0.92602056


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 4826.59it/s]

(554, 2048) (554,)
PCA for prompts: ['gpt_3_yes_no', 'should_assume']


model:bigscience-T0_3B; module:encoder; layer:1; n_components: 1; variance explained: [0.27057573]
0.27057573
model:bigscience-T0_3B; module:encoder; layer:1; n_components: 2; variance explained: [0.27057573 0.1344223 ]
0.40499803
model:bigscience-T0_3B; module:encoder; layer:1; n_components: 3; variance explained: [0.27057573 0.13442223 0.06982672]
0.47482467
model:bigscience-T0_3B; module:encoder; layer:1; n_components: 4; variance explained: [0.27057582 0.13442232 0.06982674 0.06634006]
0.5411649


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 4905.25it/s]

(554, 2048) (554,)
PCA for prompts: ['gpt_3_yes_no', 'should_assume']


model:bigscience-T0_3B; module:encoder; layer:2; n_components: 1; variance explained: [0.98539364]
0.98539364
model:bigscience-T0_3B; module:encoder; layer:2; n_components: 2; variance explained: [0.98539364 0.00834502]
0.99373865
model:bigscience-T0_3B; module:encoder; layer:2; n_components: 3; variance explained: [0.98539364 0.00834502 0.00127365]
0.9950123
model:bigscience-T0_3B; module:encoder; layer:2; n_components: 4; variance explained: [9.8539364e-01 8.3450228e-03 1.2736472e-03 8.5487531e-04]
0.99586713
